# Classical (Monte Carlo) Version of Fixed Strike Lookback Call Option

Approximating lookback option payoff using multiple vanilla european calls at different time steps.

### Call Version:

In [3]:
# Classical Version w/out using new idea
# (this is a sanity check with Qiskit code, quantum lookback version implemented in other notebooks)
# Adapted from: https://www.codearmo.com/blog/pricing-options-monte-carlo-simulation-python
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# NOTE: values below chosen to reflect small values for ease of comparison with quantum circuit
def get_price_call(T, S=2.0, K=1.896, r=0.0, q=0.0, sigma=0.4, steps=100, N=100000):
    """
    Inputs
    #S = Current stock Price
    #K = Strike Price
    #T = Time to maturity 1 year = 1, 1 months = 1/12
    #r = risk free interest rate
    #q = dividend yield
    # sigma = volatility 
    
    Output
    # [steps,N] Matrix of asset paths 
    """
    dt = T / steps
    ST = np.log(S) +  np.cumsum(((r - q - (sigma**2)/2)*dt + sigma * np.sqrt(dt) * np.random.normal(size=(steps, N))), axis=0)
    paths = np.exp(ST)
    payoffs = np.maximum(paths[-1]-K, 0)
    option_price = np.mean(payoffs)*np.exp(-r*T)
    return option_price

In [4]:
# Run over the time steps, get maximum payoff
# Let's say 40 days, w/ time step = 1 day
dt = 1 / 365
t = dt
results = []
for i in range(40):
    price = get_price_call(t)
    results.append(price)
    t+=dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

100%|██████████| 40/40 [00:17<00:00,  2.23it/s]

Lookback Option Expected Payoff: 0.1637881820120768
Lookback Option Day of Max Payoff: 40


In [7]:
# Same as before, but now we vary the volatility for a pair of consecutive days
dt = 1 / 365
t = dt
results = []
for i in range(40):
    if i == 20 or i == 21:
        price = get_price_call(t, sigma=0.9)
    else:
        price = get_price_call(t)  # Still 40% vol
    results.append(price)
    t+=dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.22668335138458265
Lookback Option Day of Max Payoff: 22


### Put Version:

In [11]:
def get_price_put(T, S=2.0, K=2.1, r=0.0, q=0.0, sigma=0.4, steps=100, N=100000):
    """
    Inputs
    #S = Current stock Price
    #K = Strike Price
    #T = Time to maturity 1 year = 1, 1 months = 1/12
    #r = risk free interest rate
    #q = dividend yield
    # sigma = volatility 
    
    Output
    # [steps,N] Matrix of asset paths 
    """
    dt = T / steps
    ST = np.log(S) +  np.cumsum(((r - q - (sigma**2)/2)*dt + sigma * np.sqrt(dt) * np.random.normal(size=(steps, N))), axis=0)
    paths = np.exp(ST)
    payoffs = np.maximum(K-paths[-1], 0)
    option_price = np.mean(payoffs)*np.exp(-r*T)
    return option_price

In [14]:
# Run over the time steps, get maximum payoff
# Let's say 40 days, w/ time step = 1 day
dt = 1 / 365
t = dt
results = []
for i in range(40):
    price = get_price_put(t)
    results.append(price)
    t+=dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.16620445270284767
Lookback Option Day of Max Payoff: 40


In [15]:
# Put version of the consecutive days difference, this time w/ vol=0.1
dt = 1 / 365
t = dt
results = []
for i in range(40):
    if i == 20 or i == 21:
        price = get_price_put(t, sigma=0.9)
    else:
        price = get_price_put(t, sigma=0.4)
    results.append(price)
    t+=dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.23372084145425698
Lookback Option Day of Max Payoff: 22
